<a href="https://colab.research.google.com/github/Liza-IITP/Dynamic_Pricing_Parking_Lots/blob/main/MODEL_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh panel --quiet

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()

In [ ]:
df = pd.read_csv("data.csv")
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

vehicle_dummies = pd.get_dummies(df["VehicleType"], prefix="VehicleType").astype(int)
for vt in ["bike", "car", "cycle", "truck"]:
    if f"VehicleType_{vt}" not in vehicle_dummies:
        vehicle_dummies[f"VehicleType_{vt}"] = 0
df = pd.concat([df, vehicle_dummies], axis=1)

weights = {"VehicleType_bike": 0.2, "VehicleType_car": 1.0, "VehicleType_cycle": 0.1, "VehicleType_truck": 1.5}
df["VehicleTypeWeight"] = sum(df[col] * wt for col, wt in weights.items())

df["TrafficLevel"] = df["TrafficConditionNearby"].map({'low': 0, 'average': 1, 'high': 2})
df["DayOfWeek"] = df["Timestamp"].dt.dayofweek
df["TimeSlot"] = df["Timestamp"].dt.hour + df["Timestamp"].dt.minute / 60

def slot_weight(t): return 1.5 if 12 <= t < 14 else (1.2 if 10 <= t < 12 or 14 <= t < 16 else (0.8 if 8 <= t < 10 else 1.0))
df["SlotWeight"] = df["TimeSlot"].apply(slot_weight)
df["ZoneWeight"] = df["Latitude"].apply(lambda lat: 1.5 if lat > 25.6 else 1.0)

df["OccRate_MA3"] = df.groupby("SystemCodeNumber")["OccupancyRate"].transform(lambda x: x.rolling(3, min_periods=1).mean())
df["OccRate_Trend"] = df.groupby("SystemCodeNumber")["OccupancyRate"].diff().fillna(0)

α, β, γ, δ, ε, τ, λ = 1.0, 0.5, 0.3, 1.5, 1.0, 0.2, 0.5
BasePrice = 10

df["RawDemand"] = (
    α * df["OccRate_MA3"] + β * df["QueueLength"] - γ * df["TrafficLevel"] +
    δ * df["IsSpecialDay"] + ε * df["VehicleTypeWeight"] + τ * df["OccRate_Trend"]
) * df["SlotWeight"] * df["ZoneWeight"]

df["NormalizedDemand"] = ((df["RawDemand"] - df["RawDemand"].min()) /
                          (df["RawDemand"].max() - df["RawDemand"].min())).clip(0, 1)

df["Model2Price"] = BasePrice * (1 + λ * df["NormalizedDemand"])
df["Model2Price_Smoothed"] = df.groupby("SystemCodeNumber")["Model2Price"].transform(lambda x: x.ewm(alpha=0.3).mean())


In [ ]:

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    return R * 2 * np.arcsin(np.sqrt(np.sin((lat2-lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2-lon1)/2)**2))

def adjust_price(row, df, radius=0.5):
    nearby = df[(df["Timestamp"] == row["Timestamp"]) & (df["SystemCodeNumber"] != row["SystemCodeNumber"])].copy()
    nearby["Distance"] = haversine(row["Latitude"], row["Longitude"], nearby["Latitude"], nearby["Longitude"])
    nearby = nearby[nearby["Distance"] <= radius]
    if nearby.empty: return row["Model2Price_Smoothed"]
    comp = nearby["Model2Price_Smoothed"].mean()
    if row["OccupancyRate"] >= 0.9 and comp < row["Model2Price_Smoothed"]: return row["Model2Price_Smoothed"] - 1
    if comp > row["Model2Price_Smoothed"] + 1: return row["Model2Price_Smoothed"] + 1
    return row["Model2Price_Smoothed"]

df["Model3Price_Competitive"] = df.apply(lambda row: adjust_price(row, df), axis=1)
df["SuggestReroute"] = df.apply(lambda r: r["OccupancyRate"] >= 0.95 and r["Model3Price_Competitive"] > 12, axis=1)

df.to_csv("parking_stream_model3.csv", index=False)

In [ ]:
class Model3Schema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    OccupancyRate: float
    QueueLength: int
    IsSpecialDay: int
    TrafficLevel: int
    DayOfWeek: int
    VehicleType_bike: int
    VehicleType_car: int
    VehicleType_cycle: int
    VehicleType_truck: int
    VehicleTypeWeight: float
    OccRate_MA3: float
    OccRate_Trend: float
    SlotWeight: float
    ZoneWeight: float
    RawDemand: float
    NormalizedDemand: float
    Model2Price_Smoothed: float
    Model3Price_Competitive: float
    SuggestReroute: bool

data = pw.demo.replay_csv("parking_stream_model3.csv", schema=Model3Schema, input_rate=1000)
fmt = "%Y-%m-%d %H:%M:%S"
data_time = data.with_columns(
    t=data.Timestamp.dt.strptime(fmt),
    day=data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

window = (
    data_time.windowby(
        pw.this.t,
        instance=(pw.this.SystemCodeNumber, pw.this.day),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    ).reduce(
        t=pw.this._pw_window_end,
        SystemCodeNumber=pw.reducers.any(pw.this.SystemCodeNumber),
        sum_price=pw.reducers.sum(pw.this.Model2Price_Smoothed),
        count_price=pw.reducers.count(pw.this.Model2Price_Smoothed),
    )
    .with_columns(
          AvgPrice_Model2 = (pw.this.sum_price / pw.this.count_price),
    )
)

def plot(source):
    fig = bokeh.plotting.figure(height=450, width=900, title="Model 3: Competitive Pricing", x_axis_type="datetime")
    fig.line("t", "AvgPrice_Model3", source=source, line_width=2, color="orange")
    fig.scatter("t", "AvgPrice_Model3", source=source, size=6, color="red")
    return fig

viz = window.plot(plot, sorting_col="t")
pn.panel(viz).servable()


In [ ]:
%%capture --no-display
pw.run()